In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from collections import defaultdict
import numpy as np
import cv2
import glob
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, precision_score


# تعريف المجلدات
folders = {
    'daisy': r'/content/drive/MyDrive/flower_photos/daisy/*.jpg',
    'dandelion': r'/content/drive/MyDrive/flower_photos/dandelion/*.jpg',
    'roses': r'/content/drive/MyDrive/flower_photos/roses/*.jpg',
    'sunflowers': r'/content/drive/MyDrive/flower_photos/sunflowers/*.jpg',
    'tulips': r'/content/drive/MyDrive/flower_photos/tulips/*.jpg'
}

# تحميل الصور والتسميات
images = []
labels = []
label_mapping = {'daisy': 0, 'dandelion': 1, 'roses': 2, 'sunflowers': 3, 'tulips': 4}

for label, folder_path in folders.items():
    for img_path in glob.glob(folder_path):
        img = cv2.imread(img_path)
        if img is not None:
            images.append(img)
            labels.append(label_mapping[label])

# دالة استخراج الميزات
def extract_features(image):
    image = cv2.resize(image, (128, 128))

    # ميزات اللون
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    color_features = cv2.mean(hsv)[:3]

    # ميزات الحواف
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 100, 200)
    edge_features = cv2.mean(edges)[:1]



    return np.concatenate([color_features, edge_features ])

# استخراج الميزات لجميع الصور
print("جاري استخراج الميزات...")
features = np.array([extract_features(img) for img in images])

# تكميم الميزات باستخدام K-means
n_clusters = 3000
print("جاري تكميم الميزات...")
kmeans = KMeans(n_clusters=n_clusters, n_init=10)
quantized_features = kmeans.fit_transform(features)

# تحويل الميزات المكممة إلى حالات منفصلة
states = np.argmin(quantized_features, axis=1)

# تقسيم الفئات إلى أربع مجموعات
group_mapping = {
    'daisy': 0,       # المجموعة 1
    'dandelion': 0,   # المجموعة 1
    'roses': 1,       # المجموعة 2
    'sunflowers': 2,  # المجموعة 3
    'tulips': 3       # المجموعة 4
}

# معلمات Q-learning
num_actions = len(label_mapping)
learning_rate = 0.1
discount_factor = 0.95
epsilon_start = 1
epsilon_end = 0.01
epsilon_decay = 0.995
episodes = 10000

# تهيئة Q-table لكل مجموعة
q_tables = {
    0: defaultdict(lambda: np.zeros(num_actions)),
    1: defaultdict(lambda: np.zeros(num_actions)),
    2: defaultdict(lambda: np.zeros(num_actions)),
    3: defaultdict(lambda: np.zeros(num_actions))
}

# التدريب باستخدام Q-learning لكل مجموعة
print("بدء التدريب...")
epsilon = epsilon_start
for episode in range(episodes):
    for state, true_label in zip(states, labels):
        # اختيار المجموعة المناسبة بناءً على الفئة
        group = group_mapping[list(label_mapping.keys())[true_label]]
        q_table = q_tables[group]

        # اختيار الحركة باستخدام epsilon-greedy
        if np.random.rand() < epsilon:
            action = np.random.randint(num_actions)
        else:
            action = np.argmax(q_table[state])

        # تشكيل المكافآت
        if action == true_label:
            reward = 2
        elif abs(action - true_label) == 1:  # فئة قريبة
            reward = 0.5
        else:
            reward = -1

        next_state = state

        # تحديث Q-table
        next_max = np.max(q_table[next_state])
        q_table[state][action] += learning_rate * (reward + discount_factor * next_max - q_table[state][action])

    # تحديث epsilon
    epsilon = max(epsilon_end, epsilon * epsilon_decay)

    if episode % 1000 == 0:
        print(f"تم إكمال الجولة {episode}, epsilon: {epsilon:.4f}")

# الاختبار
print("جاري الاختبار...")
y_pred = []
for state, true_label in zip(states, labels):
    group = group_mapping[list(label_mapping.keys())[true_label]]
    q_table = q_tables[group]
    action = np.argmax(q_table[state])
    y_pred.append(action)

accuracy = accuracy_score(labels, y_pred)
precision = precision_score(labels, y_pred, average='weighted')

print(f'الدقة: {accuracy * 100:.2f}%')
print(f'الضبط: {precision * 100:.2f}%')

# طباعة بعض الإحصائيات الإضافية
for group, q_table in q_tables.items():
    print(f'عدد الحالات الفريدة للمجموعة {group}: {len(q_table)}')
    print(f'متوسط قيم Q للمجموعة {group}: {np.mean([np.max(q) for q in q_table.values()])}')


جاري استخراج الميزات...
جاري تكميم الميزات...
بدء التدريب...
تم إكمال الجولة 0, epsilon: 0.9950
تم إكمال الجولة 1000, epsilon: 0.0100
تم إكمال الجولة 2000, epsilon: 0.0100
تم إكمال الجولة 3000, epsilon: 0.0100
تم إكمال الجولة 4000, epsilon: 0.0100
تم إكمال الجولة 5000, epsilon: 0.0100
تم إكمال الجولة 6000, epsilon: 0.0100
تم إكمال الجولة 7000, epsilon: 0.0100
تم إكمال الجولة 8000, epsilon: 0.0100
تم إكمال الجولة 9000, epsilon: 0.0100
جاري الاختبار...
الدقة: 98.12%
الضبط: 98.12%
عدد الحالات الفريدة للمجموعة 0: 1375
متوسط قيم Q للمجموعة 0: 39.29868104149711
عدد الحالات الفريدة للمجموعة 1: 621
متوسط قيم Q للمجموعة 1: 39.9999999999992
عدد الحالات الفريدة للمجموعة 2: 622
متوسط قيم Q للمجموعة 2: 39.9536517321572
عدد الحالات الفريدة للمجموعة 3: 758
متوسط قيم Q للمجموعة 3: 39.99999999341778


In [ ]:
import numpy as np
from collections import defaultdict
import cv2
import glob
from sklearn.metrics import accuracy_score, precision_score

# تعريف المجلدات
folders = {
    'daisy': r'/content/drive/MyDrive/flower_photos/daisy/*.jpg',
    'dandelion': r'/content/drive/MyDrive/flower_photos/dandelion/*.jpg',
    'roses': r'/content/drive/MyDrive/flower_photos/roses/*.jpg',
    'sunflowers': r'/content/drive/MyDrive/flower_photos/sunflowers/*.jpg',
    'tulips': r'/content/drive/MyDrive/flower_photos/tulips/*.jpg'
}

# تحميل الصور وتحويلها إلى ميزات بسيطة
def load_images(folder_path, label):
    images = []
    for img_path in glob.glob(folder_path):
        img = cv2.imread(img_path)
        if img is not None:
            img_resized = cv2.resize(img, (250, 250))  # حجم الصورة لتسريع العمليات
            hsv = cv2.cvtColor(img_resized, cv2.COLOR_BGR2HSV)
            feature_vector = hsv.flatten()[:70]  # ميزات اللون البسيطة
            images.append((feature_vector, label))
    return images

# تحميل مجموعة البيانات
data = []
label_mapping = {label: idx for idx, label in enumerate(folders.keys())}
for label, folder in folders.items():
    data.extend(load_images(folder, label_mapping[label]))

# استخراج الميزات والتسميات
features, labels = zip(*data)
features = np.array(features)
labels = np.array(labels)



# إعداد معلمات Q-learning
num_states = features.shape[1]  # عدد الميزات هو عدد الحالات
num_actions = len(label_mapping)  # عدد الفئات
learning_rate = 0.1
discount_factor = 0.95
epsilon_start = 0.9
epsilon_end = 0.1
epsilon_decay = 0.995
episodes = 1000

# تهيئة جدول Q
q_table = defaultdict(lambda: np.zeros(num_actions))

# دالة لاختيار الإجراء بناءً على السياسة الحالية
def choose_action(state, epsilon):
    if np.random.rand() < epsilon:  # الاستكشاف
        return np.random.randint(num_actions)
    else:  # الاستغلال
        return np.argmax(q_table[state])

# دالة لتحديث جدول Q
def update_q_table(state, action, reward, next_state):
    best_next_action = np.argmax(q_table[next_state])
    td_target = reward + discount_factor * q_table[next_state][best_next_action]
    td_error = td_target - q_table[state][action]
    q_table[state][action] += learning_rate * td_error

# التدريب باستخدام Q-learning
epsilon = epsilon_start
for episode in range(episodes):
    for feature, true_label in zip(features, labels):
        state = tuple(np.round(feature, 2))  # تقريب الميزات لتبسيط الجدول
        action = choose_action(state, epsilon)
        reward = 2 if action == true_label else -1
        next_state = state  # الحالة التالية هي نفسها لأن البيانات ثابتة
        update_q_table(state, action, reward, next_state)

    # تحديث epsilon
    epsilon = max(epsilon_end, epsilon * epsilon_decay)

    # عرض تقدم التدريب
    if episode % 100 == 0:
        print(f"Episode {episode}: epsilon = {epsilon:.3f}")

# الاختبار
y_pred = []
for feature in features:
    state = tuple(np.round(feature, 2))
    action = np.argmax(q_table[state])
    y_pred.append(action)

# قياس الأداء
accuracy = accuracy_score(labels, y_pred)
precision = precision_score(labels, y_pred, average='weighted')
print(f'Accuracy: {accuracy * 100:.2f}%')
print(f'Precision: {precision * 100:.2f}%')



Episode 0: epsilon = 0.895
Episode 100: epsilon = 0.542
Episode 200: epsilon = 0.329
Episode 300: epsilon = 0.199
Episode 400: epsilon = 0.121
Episode 500: epsilon = 0.100
Episode 600: epsilon = 0.100
Episode 700: epsilon = 0.100
Episode 800: epsilon = 0.100
Episode 900: epsilon = 0.100
Accuracy: 97.06%
Precision: 97.21%
